[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JamesFergusson/Introduction-to-Research-Computing/blob/master/7_BestPractice_Optimisation.ipynb)

# Validating you code

Once you have written some initial code from your prototype with good documentation and a nice modular structure (and of course carefully tracked by git) you will need to check that it is correct.  There are a few things you can do to make this much easier:

- Debug
- Error trapping
- Unit testing

For the first Python has a good inbuilt debugger that will help you, `pdg`

## Debugging
If you run code that has raised an exception in jupyter or ipython you can use the magic command `%debug` to launch an interactive debugger where you can examine the code line by line.  Here are the most useful commands

| Command | Desciption |
|---|---|
| u | Up |
| d | Down|
| p | Print |
| q | Quit |

This is best seen in an example:

In [ ]:
def bottom_func(x):
    y = x**2
    return str(x) + " squared equals " + str(y)

def top_func(z):
    result = bottom_func(z)
    return result
        
top_func('7')

In [ ]:
%debug

This can be turned on automatically with `%pdb on` so whenever an exception is raised the debugger is launched automatically

You can also run the code interactivly line by line using `%run -d` which can be more useful if your code is just wrong rather than breaking.  If you launch code with this then you can step through it with the following commands:

| Command | Desciption |
|---|---|
| n | Next line |
| s | Step into function|
| c | Continue to run normally |
| q | Quit |




In [ ]:
%run -d Examples/UnitTesting/simple.py

You can also run the debugger from the command line with:

In [ ]:
>> python3 -m pdb myscript.py

## Error trapping

It is a good idea to write you code so that when errors occours the code reports this to you.  Python is pretty good a providing reaonable error messages but it's good practice to make you own.  One simeple place to check things is whether the input makes sense. So for fibonacci we could add: 

In [ ]:
def fibonacci(num):
    
    if not isinstance(num,int):
        print("non-integer input given to function fibonacci; num="+str(num))
        return 0
    
    a=0
    b=1
    for i in range(num):
        a,b = b,a+b
    return a

fibonacci(3.4)
fibonacci(3)



You should also check error codes that come back from routines and report on them, for instance reading a file. Better examples are checking inputs are in valid ranges for functions you've created or other things which won't cause python to crash but will mean you code gives the wrong answer, for example:

In [ ]:
def harmonic_mean(list):
    x = 0e0
    for item in list:
        if item==0:
            print("Harmonic mean does not exist for data containing zeros.  Returned 'None'")
            return None
        
        x+= 1e0 / item
    x = len(list)/x
    
    return x

list1 = [1,2,3,4,5,6,7,8,9]
list2 = [1,2,3,4,5,6,7,8,9,0]
print(harmonic_mean(list1))
print(harmonic_mean(list2)) 

## Unit testing

The best way to debug your code is to catch them before they happen which you can do with unit testing.  The ideas is that you would set up a bunch of tests for the code then everytime you do a commit to master or after doing major edits you run them to check you haven't broken anything.  For the code calculating the integral in the previous notebook you may want to to create tests that check you polynomials are OK (by checking orthonormality) or that the final integral with Gauss-Legendre quadriture is correct for large l (ie set X = $\delta_{l,200}$ and see if you get the correct answer, 0.000018285996687338485).  

Having set up these tests you can then automate them to create a test package that runs, say, before a push to a central repository.  This is standard practise in commercial development enviroments and if you can include them in interview test questions this will put you above the majority of applicants.

It's a good idea to get into the habit of adding them for functions, ideally before you write it.  Then you can use them to check your code does what you thought it should.  You will end up spending lots of time checking your code when you are trying to fix bugs so setting up the tests in advance can save alot of time. Luckily in python basic ones are easy to do, you can just add it to the docstring

In [ ]:
import doctest

def function(x):
    """
    Calculate x + 2
    >>> function(5)
    7
    """
    return x+3

doctest.testmod()

This is fine for super simple tests but isn't much use once you write functions that process data rather than just a number.  There are alot of packages available but `pytest` is the standard (which is not to say it's the best).  This runs from the terminal and looks for any functions with the name `test_somefunction` or `somefunction_test` then runs them.  These functions should contain some code to run then tests to apply to the outputs using the command `assert` which accepts any booleian argument.  If our function was:

In [ ]:
def addtwo(x):
    """
        Add 2 to x
    """
    return x+1

Then our test could look like:

In [ ]:
def test_addtwo():
    """
        Test addtwo
    """
    assert( addtwo(3)==5)

These are in the files simple.py in the directory `Examples/UnitTesting`. We can test them from the command line using

In [ ]:
%%bash
pytest Examples/UnitTesting/simple.py

The tests can also be held in seperate files like test_simple.py:

In [ ]:
%%bash
pytest Examples/UnitTesting/test_simple1.py

One thing to remember is that floating point arithmitic is not exact so the test of add02 in test_simple2.py fails

In [ ]:
%%bash
pytest Examples/UnitTesting/test_simple2.py

To fix this pytest had a function called `approx` which by default allows a relative tolerance of 1e-6 which is mostly fine and it works on most data objects:

In [ ]:
from pytest import approx
import numpy as np
print(0.1 + 0.2 == approx(0.3))
print((0.1 + 0.2, 0.2+0.4) == approx((0.3,0.6)))
print({'a': 0.1 + 0.2, 'b': 0.2 + 0.4} == approx({'a': 0.3, 'b': 0.6}))
print(np.array([0.1, 0.2]) + np.array([0.2, 0.4]) == approx(np.array([0.3, 0.6])))
print(np.array([0.1, 0.2]) + np.array([0.2, 0.1]) == approx(0.3))

However if you are testing things near zero relative tolerances are useless.  Luckly `approx` also allows you to change the tolerances and make them relative or absolute.  If you specify both, it is true if either are satisfied.

In [ ]:
from pytest import approx
print(1.0001 == approx(1))
print(1.0001 == approx(1, rel=1e-3))
print(1.0001 == approx(1, abs=1e-3))
print(1.0001 == approx(1, rel=1e-5, abs=1e-3))

You can also specify a specific failure message with `fail` ie:

In [ ]:
from pytest import fail
def test_something():
    x = somefunc()
    if x in badthings:
        fail('A bad thing came back from somefunc()')

Finally note that if we specify no arguments `pytest` looks for files names `test_fimename` or `filename_test` and runs them.  In general is is best practice to put your source code and you tests in different directories as it keeps them seperate and safe.  You should have one test file for each module.  Then running pytest in the test directory will check all your code or you can run on individual modules if you want.

In [ ]:
%%bash
pytest Examples/UnitTesting/

### Exercise: Game of life

Write a small module that runs Conway's game of life, https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life.  You start with a 2D grid (whose size is to be specified at runtime) where the cells are either 'alive' or 'dead'.  Then the rules for stepping in time are:

- Overpopulation. Live cells with more than 3 neighbours die
- Underpopulation. Live cells with less than 2 neighbours die
- Reproduction. Dead cells with exactly 3 neighbours become live

Boundaries are periodic

The goal is to:
1. Prototype 
    - It should take a starting set of cells or generate a random board
    - Have a "step" function which advanced the board one timestep
    - Have a "run" function which evolves the game n steps and plots/animates the result
2. Design unit tests (The "Oscillators" from the wikipedia page are good for this)
3. THEN! Write the code, try to use debug and include error traps.  You can use your unit tests to validate as you go..

You can create all the logic yourself but the following functions will make it much easier: 2D convolution from https://docs.scipy.org/doc/scipy-0.15.1/reference/signal.html, which will count your neighbours, and then https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html for implementing the rules.